In [1]:
from pyscf import gto, scf, cc

a = 2 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"O {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.RHF(mol)
mf.kernel()

mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)


nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Tue Jan 20 10:05:29 2026
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bo

np.float64(-0.10735536052058404)

In [2]:
mol.nao

10

In [3]:
from pyscf import mcscf
nelec = 8
ncas = 6
mc = mcscf.CASSCF(mf, ncas, nelec)
mc.mo_coeff = mf.mo_coeff
# mc.frozen = frozen
nelec = mc.nelecas
h1e, enuc = mc.get_h1eff()
print(h1e)
print(enuc)

[[-4.82044209e+00 -8.59092061e-16 -3.26128013e-16 -6.21724894e-15
   6.38378239e-16  2.19668228e-16]
 [-8.59092061e-16 -4.76089213e+00  4.18897046e-14 -6.34387609e-16
   1.88516104e-15 -8.88178420e-15]
 [-3.19189120e-16  4.18897046e-14 -4.82044209e+00 -3.88578059e-16
  -6.21724894e-15  1.31775520e-15]
 [-6.43929354e-15 -6.34387609e-16 -3.60822483e-16 -4.43713775e+00
   5.48172618e-16  2.34779284e-17]
 [ 6.38378239e-16  1.88516104e-15 -6.88338275e-15  5.48172618e-16
  -4.43713775e+00  3.12825125e-16]
 [ 2.19668228e-16 -9.10382880e-15  1.31775520e-15  2.34779284e-17
   3.12825125e-16 -4.21665511e+00]]
-127.71932111492498


In [6]:
import numpy as np

def get_veff(mf, dm):
    mol = mf.mol
    vj, vk = mf.get_jk(mol, dm, hermi=1)
    return 2*vj - vk

def h1e_ras(mf, mo_coeff, ncas, ncore):
    '''
    effective one-electron integral for restricted active space
    ncas = nact_electron/2
    ncore = ncore_electrons/2
    '''
    # note casci undo DF

    mo_core = mo_coeff[:,:ncore]
    mo_cas = mo_coeff[:,ncore:ncore+ncas]

    hcore = mf.get_hcore()
    energy_core = mf.energy_nuc()
    if mo_core.size == 0:
        corevhf = 0.
    else:
        core_dm = np.dot(mo_core, mo_core.T)
        corevhf = get_veff(mf, core_dm)
        energy_core += 2 * np.einsum('ij,ji', core_dm, hcore)
        energy_core += np.einsum('ij,ji', core_dm, corevhf)
    h1eff = mo_cas.T @ (hcore+corevhf) @ mo_cas
    return h1eff, energy_core

In [7]:
h1eff, energy_core = h1e_ras(mf, mf.mo_coeff, ncas=6, ncore=4)
print(h1eff)
print(energy_core)

[[-4.82044209e+00 -8.59092061e-16 -3.26128013e-16 -6.21724894e-15
   6.38378239e-16  2.19668228e-16]
 [-8.59092061e-16 -4.76089213e+00  4.18897046e-14 -6.34387609e-16
   1.88516104e-15 -8.88178420e-15]
 [-3.26128013e-16  4.18897046e-14 -4.82044209e+00 -3.88578059e-16
  -7.10542736e-15  1.31775520e-15]
 [-6.43929354e-15 -6.34387609e-16 -3.74700271e-16 -4.43713775e+00
   5.27355937e-16  2.34779284e-17]
 [ 6.38378239e-16  1.88516104e-15 -7.54951657e-15  5.34294831e-16
  -4.43713775e+00  3.12825125e-16]
 [ 2.19668228e-16 -9.10382880e-15  1.31775520e-15  2.34779284e-17
   3.12825125e-16 -4.21665511e+00]]
-127.71932111492498


In [9]:
print(abs(h1e - h1eff).max())

8.881784197001252e-16
